# Breaking Down the Analytics Starter Dataset

Welcome to the "Getting Started" guide for Vodafone Analytics on the Developer Marketplace. In this guide, we'll demonstrate how to quickly visualise and interpret a [dataset](LINK) generated by Vodafone’s Analytics APIs. By exploring this example, you'll see the powerful insights that can be derived from data produced by our advanced analytics tools. 

The dataset file reports footfall density at the population level, meaning it doesn’t just count devices in the area. Instead, it employs sophisticated algorithms to estimate the total number of people passing through at that time, so there’s no need to try and adjust these numbers yourself - they’ve already been expanded for you. Importantly, the data is presented in an aggregated format, ensuring that it is anonymised to protect individual privacy. 

For this type of analysis, quadkeys are frequently used. If you’re not familiar with them, think of quadkeys as a sequence of digits that represent different size tiles on Earth’s surface. You can find some excellent resources on this topic in articles by [Microsoft](https://learn.microsoft.com/en-us/bingmaps/articles/bing-maps-tile-system) and [MapBox](https://labs.mapbox.com/what-the-tile/). 

The starter dataset is provided in industry-standard export format: a simple text/csv (comma separated values) file. This ensures compatibility with most common software packages. In this guide we’ll focus on the “footfall-measures” file, which provides a count of unique visitors across three London boroughs in July 2024. 

# Objective 

In this article we will: 

* Explore the raw data file. 

* Create an aggregated set of values per tile. 

* Map the busiest locations. 

Let’s get started! 

# Pre-Requisites

The analysis of datasets can be done using various tools across different technologies, all of which are effective. Given the small size of the starter dataset, the following will suffice: 

* Python 3.11
* Pip 22.4
* And the 'Starter' dataset downloaded from https://developer.vodafone.com/ and unzipped into your working directory (this guide is using 'footfall-measures_20240701_20240731_starter.csv')

If you have access to a larger dataset (e.g., national data or multiple months), you might want to consider using PySpark or a Big Data platform, or an appliance running Elastic MapReduce (EMR). 

# Setup

To begin, we will create and activate a virtual environment, install Jupyter Notebooks, and then install the necessary libraries. 

 
```#Create a virtual environment....```

```python3 -m venv vfa```

 ```#Run the activation script to configure the shell environment```

```source vfa/bin/activate ```

 ```#Install using pip3 the jupyter notebook```

```pip3 install notebook ```

 ```#Install the required libraries....```

```pip3 install pandas geopandas mercantile shapely matplotlib contextily folium plotly```

 ```#Run jupyter notebooks```

```jupyter notebook```






# Exploratory Data Analysis

Now, we need to open the file and examine its size and shape. The dataset is divided into quadkeys level 14 tiles. Since quadkeys can have leading zeros, it's crucial to load them as strings rather than numeric fields to ensure accurate location representation. After loading, we will convert the “measure_value” column to a numeric format. 




In [ ]:
import pandas as pd
import geopandas as gpd
import mercantile
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
import contextily as ctx
import folium
from folium.plugins import HeatMap
import plotly.express as px

# Specify the path to your CSV file
CSV_FOOTFALL_FILE = './footfall-measures_20240701_20240731_starter.csv'
CSV_FOOTFALL_AGG_FILE = './footfall-measures_20240701_20240731_starter-agg.csv'

EXPORT_HEATMAP_FILE = './vfa_heatmap.html'
EXPORT_CHOROPLETH_FILE = './vfa_choropleth.html'

# Read the CSV file into a DataFrame
# Set dtype to str for all columns except 'measure_value'
df = pd.read_csv(CSV_FOOTFALL_FILE, dtype=str)
df['measure_value'] = pd.to_numeric(df['measure_value'])


The output should look like this: 

In [ ]:
# Display the first few rows of the DataFrame
print(df.head())

The “dst_unit” field indicates the measurement unit for a particular destination ID, in this case, quadkey 17 (QK17), which measures approximately 200m x 200m. This measurement can vary slightly depending on the country. 

The starter file contains a measure called “unique_visitors,” meaning that when a device enters an area multiple times, it is only counted once. This field can contain other values which represent different measurements being provided, unique visitor count is the most popular measure.  The “time_aggregation” field represents the period under study, which for this dataset is July 2024. 

To explore the time dimensions included in the file, use the following code: 

In [ ]:
# How many time aggregations are in the file 
print ( 'There are ', df['time_aggregation'].nunique(), 'time dimensions in this file.' ) 

# Show the first 5 entries 
df['time_aggregation'].unique()[:5] 

These dimensions represent aggregates for each hour and day in July 2024.  775 is the number of hours in the month (31*24=744) added to the number of daily values (31). We will add a measure_granularity field (daily or hour) derived from the time_aggregation field.  Then we will remove the daily values and work with the hourly values (744 rows).

In [ ]:
# Infer daily or hourly measurement (daily are yyyy-mm-dd = 10 characters)
df['measure_granularity'] = df['time_aggregation'].apply(
    lambda x: 'daily' if len(x) == 10 else 'hour'
)

# Filter out daily values
df = df[df['measure_granularity'] != 'daily']

# How many time aggregations do we now have (expect 744)
print ( 'There are ', df['time_aggregation'].nunique(), 'time dimensions now available.' ) 

Given its size, this dataset can be quite demanding when processed on a laptop. The steps below will create an aggregate profile for each tile and save it in a different file, which will be used for initial analysis. 

<b>Paragraph change</b>
The code snippet below calculates the minimum, maximum, and average footfall over the entire month for each quadkey.  Note you can not sum the hourly values together to create a daily.  Imagine you have the same people at the location for the entire 24 hour period, summing the values would give you the wrong value and so we provide the daily unique visitor count in the dataset.


In [ ]:
df_agg = df.groupby('dst_id', as_index=False).agg(
    measure_min=('measure_value', 'min'),
    measure_max=('measure_value', 'max'),
    measure_avg=('measure_value', 'mean')
)

# Convert the mean average to a whole number
df_agg['measure_avg'] = df_agg['measure_avg'].round(0)

# Save the file to disk
df_agg.to_csv(CSV_FOOTFALL_AGG_FILE, index=False)

## Identifying and Plotting Top Locations 

Using the file generated by the previous code, the following steps will extract the top 10 locations and plot them on a map. 

To begin, some helper functions are needed when dealing with quadkeys. The functions below create the polygon/bounding box for the tiles and provide the latitude and longitude located at the central point of the tile: 

In [ ]:
# Helper function to calculate the quadkey boundary, we will use it later for plotting on maps

# Function to calculate the centroid of a quadkey
def quadkey_to_lat_lon(quadkey):
    # Convert quadkey to tile
    tile = mercantile.quadkey_to_tile(quadkey)
    # Get the bounding box of the tile
    bounds = mercantile.bounds(tile)
    # Calculate the center of the tile
    lat = (bounds.north + bounds.south) / 2
    lon = (bounds.east + bounds.west) / 2
    return lat, lon
    

# Function to convert a quadkey to a polygon
def quadkey_to_polygon(quadkey):
    # Convert quadkey to tile coordinates
    tile = mercantile.quadkey_to_tile(quadkey)
    # Get bounding box for the tile
    bounds = mercantile.bounds(tile)
    # Create a polygon from the bounding box
    return Polygon([
        (bounds.west, bounds.south),  # Bottom left
        (bounds.west, bounds.north),  # Top left
        (bounds.east, bounds.north),  # Top right
        (bounds.east, bounds.south),  # Bottom right
        (bounds.west, bounds.south)   # Close polygon
    ])

# Function to calculate the best map zoom level given a dataframe of latitude and longitude points
def map_zoom_level(df):
    # Calculate the range between map locations
    lat_range = df['latitude'].max() - df['latitude'].min()
    lon_range = df['longitude'].max() - df['longitude'].min()
    
    #Determine a suitable zoom level, adjusting the constant factor as needed
    if max(lat_range, lon_range) < 1:
        zoom_level = 10  # Higher zoom level for small area
    elif max(lat_range, lon_range) < 5:
        zoom_level = 6  # Medium zoom level for medium area
    else:
        zoom_level = 4  # Lower zoom level for large area
    
    return zoom_level+1

Now, calculate the centroid latitude and longitude and store them in the dataset along with the geometry of the quadkey (a polygon). These will be used later when the polygons are plotted on maps. 

In [ ]:
# Create the extra supporting columns used for mapping
df_agg[['latitude', 'longitude']] = df_agg['dst_id'].apply(quadkey_to_lat_lon).apply(pd.Series)

# Creating a new column with geometry data
df_agg['geometry'] = df_agg['dst_id'].apply(quadkey_to_polygon)

Next, identify the top 10 tiles by average footfall and maximum footfall: 

In [ ]:
# Top 10 busiest tiles by their average footfall
df_top_10_avg = df_agg.sort_values(by='measure_avg', ascending=False).head(10)

# Top 10 busiest tiles by their maximum footfall
df_top_10_max = df_agg.sort_values(by='measure_max', ascending=False).head(10)

In [ ]:
# View the dataframe - Expected output below
df_top_10_avg

In [ ]:
# View the dataframe - Top 10 by maximum footfall - Expected output below
df_top_10_max

## Plotting quadkeys on a map 

Simply looking at a quadkey ID does not provide much information about its location. In this next step, we will plot both the top 10 locations for average and maximum footfall on a map. To do this, the top 10 dataframes will be converted into GeoDataFrames. The [EPSG:4326](https://epsg.io/4326) (also known as WGS84) coordinate system will be used. This system is commonly used in online maps.  

This map will display the top 10 average and maximum footfall locations: 

In [ ]:
gdf_avg = gpd.GeoDataFrame(df_top_10_avg, geometry='geometry', crs='EPSG:4326')
gdf_max = gpd.GeoDataFrame(df_top_10_max, geometry='geometry', crs='EPSG:4326')

# Plot the polygons on a map
fig, ax = plt.subplots(figsize=(10, 10))

# Add the geo dataframes to the plot (blue=avg, red=max)
gdf_avg.plot(ax=ax, color='blue', edgecolor='black', alpha=0.5)
gdf_max.plot(ax=ax, color='red', edgecolor='black', alpha=0.5)

# Add some labelling
plt.title('Busy locations by Maximum and Average footfall')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Optionally, add a basemap for better visualization
try:
    # Use OpenStreetMap as a basemap provider
    ctx.add_basemap(ax, crs=gdf_avg.crs, source=ctx.providers.OpenStreetMap.Mapnik)
except ImportError:
    print("contextily is not installed, skipping basemap.")

plt.show()

<center><i> Figure 1: Plotted quadkeys for the top 10 average and maximum footfall</i></center>

# Summary 

Congratulations! You’ve completed this guide for exploring and mapping the Vodafone Analytics starter dataset. Starting from the raw file, we went over the meaning of them, how to trace quadkeys into polygons and finally, how to plot a subset of them in a map. On a next installment, we will dive further into analysis of the dataset and go over a few additional information that can be extracted from it.  

Be sure to check out future guides and blog posts to see how other companies and academia are using these valuable data sources. 

Until then, enjoy your coding. 